In [ ]:
%load_ext google.cloud.bigquery

import numpy as np
import pandas as pd

#### Fake dataset 1 - prosty graf

In [ ]:
fake1 = pd.DataFrame([
     {'prev':'START', 'next':'A', 'prob':0.3}
    ,{'prev':'START', 'next':'B', 'prob':0.7}
    ,{'prev':'A', 'next':'C', 'prob':0.2}
    ,{'prev':'A', 'next':'D', 'prob':0.8}
    ,{'prev':'B', 'next':'D', 'prob':1.0}
    ,{'prev':'C', 'next':'N_CONV', 'prob':1.0}
    ,{'prev':'D', 'next':'N_CONV', 'prob':0.1}
    ,{'prev':'D', 'next':'CONV', 'prob':0.9}
])

print(fake1)

In [62]:
class Crawler:
    graph = pd.DataFrame()
    active_crawlers = []
    paths = []
    current_index = 0
    
    def __init__(self, history):
        if Crawler.graph.empty:
            print('Before using Crawler you must load a graph! (Crawler.load_graph())')
        else:
            Crawler.active_crawlers.append(self)
            self.index = Crawler.current_index
            Crawler.current_index += 1
            self.history = history
            print('Created crawler{}, active crawlers: {} ({})'.format(self.index, len(Crawler.active_crawlers), self.history))
    
    def _step(self):
        current_node = self.history[-1]
        exits = Crawler.graph[Crawler.graph['prev'] == current_node]['next']
        if exits.empty:
            print('\t\tCrawler{} finished'.format(self.index))
            print('\t\tCrawler{}\'s path: {} '.format(self.index, self.history))
            if current_node in ['CONV', 'N_CONV']:
                Crawler.paths.append(self.history)
        else:
            for e in exits:
                new = Crawler(self.history.copy())
                new.history.append(e)
                new._step()
        idx = self.index
        Crawler.active_crawlers.remove(self)
        print('Removed crawler{}, active crawlers: {}'.format(idx, len(Crawler.active_crawlers)))
    
    @staticmethod
    def start():
        c0 = Crawler(['START'])
        c0._step()
            
    @staticmethod
    def load_graph(graph):
        Crawler.graph = graph
    

In [63]:
Crawler.load_graph(fake1)
Crawler.start()

Created crawler0, active crawlers: 1 (['START'])
Created crawler1, active crawlers: 2 (['START'])
Created crawler2, active crawlers: 3 (['START', 'A'])
Created crawler3, active crawlers: 4 (['START', 'A', 'C'])
		Crawler3 finished
		Crawler3's path: ['START', 'A', 'C', 'N_CONV'] 
Removed crawler3, active crawlers: 3
Removed crawler2, active crawlers: 2
Created crawler4, active crawlers: 3 (['START', 'A'])
Created crawler5, active crawlers: 4 (['START', 'A', 'D'])
		Crawler5 finished
		Crawler5's path: ['START', 'A', 'D', 'N_CONV'] 
Removed crawler5, active crawlers: 3
Created crawler6, active crawlers: 4 (['START', 'A', 'D'])
		Crawler6 finished
		Crawler6's path: ['START', 'A', 'D', 'CONV'] 
Removed crawler6, active crawlers: 3
Removed crawler4, active crawlers: 2
Removed crawler1, active crawlers: 1
Created crawler7, active crawlers: 2 (['START'])
Created crawler8, active crawlers: 3 (['START', 'B'])
Created crawler9, active crawlers: 4 (['START', 'B', 'D'])
		Crawler9 finished
		Cra

In [64]:
Crawler.paths

[['START', 'A', 'C', 'N_CONV'],
 ['START', 'A', 'D', 'N_CONV'],
 ['START', 'A', 'D', 'CONV'],
 ['START', 'B', 'D', 'N_CONV'],
 ['START', 'B', 'D', 'CONV']]